In [6]:
import pandas as pd
import numpy as np
np.random.seed(123)

In [20]:
r_df = pd.read_csv(f'./data/anime_ratings__500000_10000.csv', delimiter=';', header=0)

In [21]:
r_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1671845 entries, 0 to 1671844
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   status     1668460 non-null  object 
 1   mal_id     1671673 non-null  float64
 2   username   1671845 non-null  object 
 3   rating     1671845 non-null  float64
 4   timestamp  1478367 non-null  object 
 5   source     1671845 non-null  object 
dtypes: float64(2), object(4)
memory usage: 76.5+ MB


In [32]:
r_df['timestamp'] =pd.to_datetime(r_df.loc[:,'timestamp'], errors='coerce', utc=True)

In [33]:
r_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1671845 entries, 0 to 1671844
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype              
---  ------     --------------    -----              
 0   status     1668460 non-null  object             
 1   mal_id     1671673 non-null  float64            
 2   username   1671845 non-null  object             
 3   rating     1671845 non-null  float64            
 4   timestamp  1478367 non-null  datetime64[ns, UTC]
 5   source     1671845 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(2), object(3)
memory usage: 76.5+ MB


In [34]:
r_df.loc[r_df['status'] == 'completed']

,status,mal_id,username,rating,timestamp,source
136,completed,38101.0,HayashiSawaryo,8.0,NaT,anilist
137,completed,11759.0,HayashiSawaryo,7.0,NaT,anilist
138,completed,22199.0,HayashiSawaryo,7.0,NaT,anilist
139,completed,22729.0,HayashiSawaryo,9.0,NaT,anilist
140,completed,27655.0,HayashiSawaryo,8.0,NaT,anilist
...,...,...,...,...,...,...
1671145,completed,15911.0,Daverost,10.0,2015-02-06 00:00:00+00:00,MAL
1671146,completed,6377.0,Daverost,9.0,2011-03-09 00:00:00+00:00,MAL
1671147,completed,7044.0,Daverost,9.0,2011-05-02 00:00:00+00:00,MAL
1671148,completed,1195.0,Daverost,9.0,2008-01-09 00:00:00+00:00,MAL


# Data Preprocessing

In [25]:
# randomly select 30% of the users
# will result in more manageable mem usage
rand_users = np.random.choice(r_df['username'].unique(), 
                                size=int(len(r_df['username'].unique())*0.3), 
                                replace=False)
ratings = r_df.loc[r_df['username'].isin(rand_users)]

In [26]:
ratings

,status,mal_id,username,rating,timestamp,source
1218,completed,38671.0,sivio,10.0,2019-12-27 00:00:00,anilist
1219,completed,38993.0,sivio,9.0,2019-12-20 00:00:00,anilist
1220,completed,31964.0,sivio,7.5,2019-12-20 00:00:00,anilist
1221,completed,38408.0,sivio,9.0,2020-06-17 00:00:00,anilist
1222,completed,11061.0,sivio,10.0,2020-04-15 00:00:00,anilist
...,...,...,...,...,...,...
1671840,planned,1840.0,Daverost,0.0,2008-12-09 00:00:00+00:00,MAL
1671841,planned,3712.0,Daverost,0.0,2009-10-09 00:00:00+00:00,MAL
1671842,planned,5764.0,Daverost,0.0,2010-01-12 00:00:00+00:00,MAL
1671843,planned,36999.0,Daverost,0.0,2018-10-11 00:00:00+00:00,MAL


## Train-Test split
We will implement our train-test split strategy using the leave-one-out methodology. For each user, the most recent review is used as the test set (i.e. leave one out), while the rest will be used as training data .

This train-test split strategy is often used when training and evaluating recommender systems. Doing a random split would not be fair, as we could potentially be using a user’s recent reviews for training and earlier reviews for testing. This introduces data leakage with a look-ahead bias, and the performance of the trained model would not be generalizable to real-world performance.

In [36]:
tst = ratings.groupby(['username'])['timestamp']

In [38]:
tst